In [1]:
import tensorflow as tf
import numpy as np
import os

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, AveragePooling1D, Conv1D, DenseFeatures, Input, Flatten, GlobalAveragePooling1D

Load all gestures from disk and label them accordingly

In [2]:
dataset_dir = "gestures"

gestures = ["Computer", "Elephant", "Sorry", "No-Sign"]

dataset_values = []
dataset_labels = []

for gesture in gestures:
    for filename in os.listdir(f"{dataset_dir}/{gesture}"):
        with open(os.path.join(f"{dataset_dir}/{gesture}", filename), 'r') as file:
            file_data = []

            text = file.read()
            lines = text.splitlines()

            for line in lines:
                int_values = []
                for value in line.split():
                    int_values.append(int(value))

                file_data.append(int_values)

            dataset_values.append(file_data)

            if gesture == "Computer":
                dataset_labels.append([1, 0, 0, 0])
            elif gesture == "Elephant":
                dataset_labels.append([0, 1, 0, 0])
            elif gesture == "Sorry":
                dataset_labels.append([0, 0, 1, 0])
            elif gesture == "No-Sign":
                dataset_labels.append([0, 0, 0, 1])
            else:
                print("You did not change the labeling of the dataset you blithering idiot")

dataset_values = np.array(dataset_values)
dataset_labels = np.array(dataset_labels)

print(dataset_values.shape)
print(dataset_labels.shape)

(1584, 200, 6)
(1584, 4)


Load numpy array into tensorflow dataset

In [5]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 2000
VALIDATION_SIZE = 100

dataset = tf.data.Dataset.from_tensor_slices((dataset_values, dataset_labels))
dataset = dataset.shuffle(SHUFFLE_BUFFER_SIZE)

train_dataset = dataset.skip(VALIDATION_SIZE).batch(BATCH_SIZE)
validation_dataset = dataset.take(VALIDATION_SIZE).batch(BATCH_SIZE)  

for values, labels in train_dataset.take(1):
    print(labels.numpy(), values.numpy())

[[1 0 0 0]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 1]] [[[-11014   6841  11574   -902    170    -33]
  [-11038   6785  11561  -1281    130    -91]
  [-11049   6650  11545  -1724     74   -126]
  ...
  [-10489   6153  12758  -1588   -177    113]
  [-10436   6013  12577  -1625   -265     69]
  [-10341   5969  12573  -1801   -536    120]]

 [[-15628  

In [11]:
model = Sequential([
    Conv1D(filters = 6, kernel_size=70, activation='relu', input_shape=(200, 6)),
    GlobalAveragePooling1D(),
    Dense(20, activation='relu'),
    Dense(4, activation='softmax')
])

model.build(input_shape=(1584, 200, 6))
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
model.summary()



model.fit(train_dataset, validation_data=validation_dataset, epochs=20)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_5 (Conv1D)           (None, 132, 4)            1660      
                                                                 
 global_average_pooling1d_5   (None, 4)                0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_5 (Dense)             (None, 4)                 20        
                                                                 
Total params: 1,680
Trainable params: 1,680
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
15/24 [=================>............] - ETA: 0s - loss: 1780.9850 - accuracy: 0.3667

2022-04-15 16:17:25.245328: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 0s 12ms/step - loss: 1283.5197 - accuracy: 0.3753 - val_loss: 238.4717 - val_accuracy: 0.5100
Epoch 2/20
16/24 [===================>..........] - ETA: 0s - loss: 158.2547 - accuracy: 0.4043

2022-04-15 16:17:25.562906: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 0s 8ms/step - loss: 152.2147 - accuracy: 0.4097 - val_loss: 26.7438 - val_accuracy: 0.3200
Epoch 3/20
24/24 [==============================] - 0s 8ms/step - loss: 74.2808 - accuracy: 0.4319 - val_loss: 106.9065 - val_accuracy: 0.4200
Epoch 4/20
24/24 [==============================] - 0s 8ms/step - loss: 50.2895 - accuracy: 0.4360 - val_loss: 150.2161 - val_accuracy: 0.3000
Epoch 5/20
24/24 [==============================] - 0s 8ms/step - loss: 33.9371 - accuracy: 0.4394 - val_loss: 56.0032 - val_accuracy: 0.4100
Epoch 6/20
24/24 [==============================] - 0s 8ms/step - loss: 34.4366 - accuracy: 0.4555 - val_loss: 24.3813 - val_accuracy: 0.4600
Epoch 7/20
24/24 [==============================] - 0s 8ms/step - loss: 21.0716 - accuracy: 0.4629 - val_loss: 18.7620 - val_accuracy: 0.5200
Epoch 8/20
24/24 [==============================] - 0s 8ms/step - loss: 25.8368 - accuracy: 0.4454 - val_loss: 6.9571 - val_accuracy: 0.3900
Epoch 9/20
24/2